ref: https://myapollo.com.tw/blog/langchain-expression-language/

# LCEL (LangChain Expression Language)

## RunnableSequence 

In [4]:
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = Ollama(model='llama3')

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

prompt_output = prompt.invoke({"input": 'Hi there'})
llm_output = llm.invoke(prompt_output)
answer = StrOutputParser().invoke(llm_output)

print(answer)

Robot: Greetings, human. I'm happy to assist you with any questions or topics you'd like to discuss. What's on your mind today?


## RunnableParallel

In [8]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def add_two(x: int) -> int:
    return x + 2


runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(add_two)

parallel = {"runnable_1": runnable_1, "runnable_2": runnable_2}

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke(1)

print(answer)

{'runnable_1': 2, 'runnable_2': 3}


In [9]:
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def add_two(x: int) -> int:
    return x + 2


runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(add_two)

parallel = RunnableParallel(r1=runnable_1, r2=runnable_2)

chain = RunnableLambda(lambda x: x) | parallel
answer = chain.invoke(1)

print(answer)

{'r1': 2, 'r2': 3}


## RunnableBranch

In [14]:
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = Ollama(model='llama3')

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

default_chain = prompt | llm # you can customize the chain here
python_chain = prompt | llm  # you can customize the chain here

def route(x):
    if 'python' in x['input']:
        return python_chain
    return default_chain

chain = RunnableLambda(route)
# print(chain.invoke({"input": "python is the best"}))
print(chain.invoke({"input": "what's you name?"}))

I'm just an AI, so I don't have a personal name. But if you want to give me one, that's okay too! Some people call me "Assistant" or "AI Assistant", while others like to give me more playful names like "Chatbot" or "Language Model". You can choose whatever name you like for me - just keep in mind that I'll always be here to help and assist you with your questions and tasks!


In [17]:
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm_llama3 = Ollama(model='llama3')
llm_mistral = Ollama(model='mistral')

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

default_chain = prompt | llm_llama3
python_chain = prompt | llm_mistral

def route(x):
    if 'python' in x['input']:
        return python_chain
    return default_chain

chain = RunnableLambda(route)
# print(chain.invoke({"input": "python is the best"}))
print(chain.invoke({"input": "who create you?"}))

I was created by a team of software developers using a range of technologies, including natural language processing (NLP) and machine learning. My creators are the researchers at Meta AI, a company that focuses on developing artificial intelligence models for various applications.

More specifically, I am based on a type of AI model called a transformer, which is designed to process and generate human-like text. This architecture was introduced in a research paper by Vaswani et al. in 2017 and has since been widely adopted in the NLP community.

My training data consists of a massive corpus of text, which I use to learn patterns and relationships in language. This corpus is sourced from various places, including books, articles, and websites, and is used to train me on how to generate human-like responses to user input.

So, while I don't have a single "creator" in the classical sense, I am the result of the collective efforts of many researchers and developers who have contributed to 

## RunnablePassthrough

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = RunnablePassthrough.assign(input=lambda x: x['input'] + ' this is important to me.') | prompt
print(chain.invoke({"input": "python is the best."}))

messages=[HumanMessage(content='python is the best. this is important to me.')]


## 輸入/輸出 (Input & Output Schema)

In [26]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

In [29]:
print(prompt.input_schema.schema())
# print(prompt.output_schema.schema())

{'title': 'PromptInput', 'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}}


In [30]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate


llm = Ollama(model='llama3')
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm
print(chain.input_schema.schema())

{'title': 'PromptInput', 'type': 'object', 'properties': {'input': {'title': 'Input', 'type': 'string'}}}


## 列印 Chain 的樣子

In [33]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate


llm = Ollama(model='llama3')
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
])

chain = prompt | llm
chain.get_graph().print_ascii()

    +-------------+    
    | PromptInput |    
    +-------------+    
           *           
           *           
           *           
+--------------------+ 
| ChatPromptTemplate | 
+--------------------+ 
           *           
           *           
           *           
      +--------+       
      | Ollama |       
      +--------+       
           *           
           *           
           *           
   +--------------+    
   | OllamaOutput |    
   +--------------+    
